### creating pseudo mutiplex from singleplex exps 

In [ ]:
import numpy as np
import os
import shutil
import json
import cv2
import glob
import sys 
root_code = os.path.dirname(os.path.dirname(os.getcwd()))
sys.path.insert(0, root_code)

from codebase.utils.constants import *


In [ ]:
results_path = '/raid/sonali/project_mvs/results/'

In [ ]:
def create_folder(path): 
    if not os.path.exists(path): 
        os.makedirs(path)

#### create pseudo folder 

In [ ]:
# create pseudo folder
# pseudo_results_name = "0jpt9ixq_dataaug-v2-flip_split3_pseudo_multiplex_no-wt_no-checkerboard"
pseudo_results_name = "1kh90kst_dataaug-v2-flip_split3_pseudo_multiplex_selected-snr-set_no-wt_no-checkerboard"

pseudo_path = os.path.join(results_path, pseudo_results_name)
create_folder(pseudo_path)

#### create args file


In [ ]:
# create args.txt file
source_path = os.path.join(results_path, '3ifhwgc0_dataaug-v2-flip_split3_Ki-67_no-wt_no-checkerboard')

source_args = os.path.join(source_path, 'args.txt')
target_args = os.path.join(pseudo_path, 'args.txt')

shutil.copy2(source_args, target_args) # source, target

In [ ]:
# change submission id and protein set
with open(source_args, "r") as jsonFile:
    data = json.load(jsonFile)
    
# data["protein_set"] = "prots_pseudo_multiplex"
data["protein_set"] = "selected_snr"
data["submission_id"] = pseudo_results_name

with open(target_args, 'w') as f:
    json.dump(data, f, indent=2)

#### create checkpoint folder and image paths 

In [ ]:
pseudo_chkpt_path = os.path.join(pseudo_path, 'chkpt_selection')
create_folder(pseudo_chkpt_path)

In [ ]:
source_chkpt = os.path.join(source_path, 'chkpt_selection', 'best_epoch-level_2-valid.txt')
target_chkpt = os.path.join(pseudo_path, 'chkpt_selection', 'best_epoch-level_2-valid.txt')
shutil.copy2(source_chkpt, target_chkpt) # copy source, target

In [ ]:
# create folders with desired epoch 
# get best epoch 
with open(target_chkpt, "r") as jsonFile:
    data = json.load(jsonFile)
    
best_epoch_multiplex = data['best_epoch']
print('best_epoch_multiplex: ', best_epoch_multiplex)

# create folders for sets and levels 
image_sets = ['test_images', 'valid_images']
levels = ['level_2', 'level_4', 'level_6']

for image_set in image_sets: 
    for level in levels: 
        create_folder(os.path.join(pseudo_path, image_set, 'epoch' + str(best_epoch_multiplex) +'-1', level))

### list of experiments that are needed for pseudo multiplex  

In [ ]:
proteins_for_pseudo_multiplex = selected_prots_snr # prots_pseudo_multiplex  
proteins_for_pseudo_multiplex

In [ ]:
# get paths of all relevant singleplex exps 
singleplex_paths = []
for protein in proteins_for_pseudo_multiplex: 
    singleplex_paths.append(glob.glob(results_path + '/*' + protein +  '_no-wt_no-checkerboard')[0])
len(singleplex_paths), singleplex_paths

In [ ]:
# get best epoch for all singleplex exps 
best_epochs = []
for singleplex_path in singleplex_paths: 
    singleplex_chkpt = os.path.join(singleplex_path, 'chkpt_selection', 'best_epoch-level_2-valid.txt')
    with open(singleplex_chkpt, "r") as jsonFile:
        data = json.load(jsonFile)
    best_epochs.append(data['best_epoch'])
best_epochs

#### cretate and visualise pseudo multiplex  

In [ ]:
def get_rois(which_set, project_path, cv_split): 
    kfold_splits = json.load(open(os.path.join(project_path, CV_SPLIT_ROIS_PATH)))
    aligns = kfold_splits[cv_split][which_set]
    return aligns 

In [ ]:
# /valid_images/epoch48-1/level_2
project_path = '/raid/sonali/project_mvs/'
cv_split = 'split3'

aligns_sets = ['test']#, 'valid']
levels = ['level_2', 'level_4', 'level_6']

# 
for align_set in aligns_sets: # for test/valid   
    print('align_set: ', align_set)
    for level in levels: # for level_2/level_4, level_6
        print('level: ', level)
        rois = get_rois(align_set, project_path, cv_split)
        for roi in rois: # for each roi in the test/valid set
            print(roi)
            
            # initialise an empty numpy array            
            roi_width = 4000//(2**int(level.split('_')[-1]))
            roi_multiplex = np.zeros((roi_width, roi_width, len(proteins_for_pseudo_multiplex)), dtype=np.float32)
            
            for i, protein in enumerate(proteins_for_pseudo_multiplex): # for each protein 
                print(protein)
                singleplex_path = singleplex_paths[i]
                best_epoch = best_epochs[i]
                print(best_epoch, singleplex_path)
                # load numpy predictions for roi for protein 
                pred_prot = np.load(os.path.join(singleplex_path, align_set + '_images', 
                                            'epoch' + str(best_epoch) + '-1', level, roi + '.npy'))
                print(pred_prot.shape, roi_multiplex.shape)
                
                roi_multiplex[:,:,i] = pred_prot[:,:,0]
                
            # save roi_multiplex in desired path                 
            roi_multiplex = np.save(os.path.join(pseudo_path, align_set + '_images', 
                            'epoch' + str(best_epoch_multiplex) + '-1', level, roi + '.npy'), roi_multiplex)


#### change folder permissions 

In [ ]:
!chgrp -R mvsgroup $pseudo_path
!chmod -R 770 $pseudo_path

#### QC to compare of numpy values match between singleplex and multiplex 

In [ ]:
align_set = 'test'
rois = get_rois(align_set, project_path, cv_split)
level = 'level_6'

for roi in rois: 
    print(roi)
    prot_multiplex = np.load(os.path.join(pseudo_path, align_set + '_images', 
                    'epoch' + str(best_epoch_multiplex) + '-1', level, roi + '.npy'))    
#     print(prot_multiplex.shape)
    for i, protein in enumerate(proteins_for_pseudo_multiplex):
        prot_singleplex = np.load(os.path.join(singleplex_paths[i], align_set + '_images', 
                                            'epoch' + str(best_epochs[i]) + '-1', level, roi + '.npy'))            
        print(np.array_equal(prot_singleplex, np.expand_dims(prot_multiplex[:,:,i], axis=2)))

prot_singleplex.shape, prot_multiplex.shape
    